In [ ]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from pydub import AudioSegment
from sklearn.model_selection import train_test_split
from collections import defaultdict
import librosa
import seaborn as sns
 

In [ ]:
TARGET_LENGTH = 30.0   
SAMPLE_RATE = 16000
AUDIO_DIR = "/kaggle/input/diarization/Dataset/data/audio"
RTTM_DIR = "/kaggle/input/diarization/Dataset/data/markups"
OUTPUT_AUDIO_DIR = "/kaggle/working/new_audio"
OUTPUT_RTTM_DIR = "/kaggle/working/new_rttm"

os.makedirs(OUTPUT_AUDIO_DIR, exist_ok=True)
os.makedirs(OUTPUT_RTTM_DIR, exist_ok=True)

# **EDA** 

In [ ]:
import pandas as pd

def parse_rttm(file_path):
    columns = ['type', 'file_id', 'channel', 'start', 'duration', 'ortho', 'stype', 'name', 'conf', 'slat']
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            if line.strip():
                parts = line.strip().split()
                data.append(parts[:len(columns)])
    return pd.DataFrame(data, columns=columns)

rttm_files = [f'/kaggle/input/diarization/Dataset/data/markups/{i}.rttm' for i in range(1, 449)]   
all_rttm = pd.concat([parse_rttm(f) for f in rttm_files])
all_rttm['start'] = all_rttm['start'].astype(float)
all_rttm['duration'] = all_rttm['duration'].astype(float)
all_rttm['end'] = all_rttm['start'] + all_rttm['duration']

In [ ]:
import librosa
import matplotlib.pyplot as plt

audio_files = [f'/kaggle/input/diarization/Dataset/data/audio/{i}.wav' for i in range(1, 449)]
durations = []
for audio_file in audio_files:
    dur = librosa.get_duration(path=audio_file)
    durations.append(dur)
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.hist(durations, bins=20)
plt.title("Распределение длительности аудиофайлов")
plt.xlabel("Длительность (сек)")
plt.ylabel("Количество файлов")
 

plt.subplot(1, 2, 2)
speakers_per_file = all_rttm.groupby('file_id')['name'].nunique()
plt.hist(speakers_per_file, bins=10)
plt.title("Количество спикеров на файл")
plt.xlabel("Число спикеров")
plt.ylabel("Количество файлов")
plt.show()



In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.hist(all_rttm['duration'], bins=50)
plt.title("Распределение длительности реплик")
plt.xlabel("Длительность (сек)")
plt.ylabel("Количество сегментов")

plt.subplot(1, 2, 2)
speaker_duration = all_rttm.groupby('name')['duration'].sum().sort_values(ascending=False)
speaker_duration.plot(kind='bar')
plt.title("Общее время речи спикеров")
plt.xlabel("Спикер")
plt.ylabel("Суммарная длительность (сек)")
plt.show()

In [ ]:
from itertools import combinations

In [ ]:
def check_overlaps(df):
    overlaps = 0
    speakers = df['name'].unique()
    for spk1, spk2 in combinations(speakers, 2):
        segments1 = df[df['name'] == spk1][['start', 'end']].values
        segments2 = df[df['name'] == spk2][['start', 'end']].values
         
        for s1 in segments1:
            for s2 in segments2:
                if max(s1[0], s2[0]) < min(s1[1], s2[1]):
                    overlaps += 1
    return overlaps

overlaps_per_file = all_rttm.groupby('file_id').apply(check_overlaps)
print(f"Среднее число наложений на файл: {overlaps_per_file.mean()}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def analyze_speaker_balance(rttm_dir):
    all_rttm = []
    for rttm_file in os.listdir(rttm_dir):
        if rttm_file.endswith('.rttm'):
            df = pd.read_csv(os.path.join(rttm_dir, rttm_file), 
                           sep=' ', header=None,
                           names=['type', 'file_id', 'channel', 'start', 
                                  'duration', 'ortho', 'stype', 'name', 
                                  'conf', 'slat'])
            all_rttm.append(df)
    
    all_rttm = pd.concat(all_rttm)
    
    # Анализ по времени речи
    speaker_stats = all_rttm.groupby('name').agg(
        total_duration=('duration', 'sum'),
        segment_count=('duration', 'count'),
        avg_duration=('duration', 'mean')
    ).sort_values('total_duration', ascending=False)
    
    # Визуализация
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 2, 1)
    speaker_stats['total_duration'].head(20).plot(kind='bar')
    plt.title('Топ-20 спикеров по времени речи')
    plt.ylabel('Секунды')
    
    plt.subplot(1, 2, 2)
    speaker_stats['segment_count'].head(20).plot(kind='bar', color='orange')
    plt.title('Топ-20 спикеров по количеству реплик')
    plt.ylabel('Количество')
    
    plt.tight_layout()
    plt.show()
    
    return speaker_stats

speaker_stats = analyze_speaker_balance(RTTM_DIR)

In [ ]:
speaker_stats

In [ ]:
speakers_per_file = all_rttm.groupby('file_id')['name'].nunique()

plt.figure(figsize=(12, 6))
sns.histplot(speakers_per_file, bins=10)
plt.title("Количество спикеров на файл")
plt.xlabel("Число спикеров")
plt.ylabel("Количество файлов")
plt.show()

print(f"Среднее количество спикеров на файл: {speakers_per_file.mean():.2f}")
print(f"Максимальное количество спикеров в одном файле: {speakers_per_file.max()}")
print(f"Минимальное количество спикеров в одном файле: {speakers_per_file.min()}")
print(f"Наиболее частое количество спикеров: {speakers_per_file.mode().values[0]}")

In [ ]:
audio_files = [f'{AUDIO_DIR}/{i}.wav' for i in range(1, 449)]
durations = []
for audio_file in audio_files:
    dur = librosa.get_duration(path=audio_file)
    durations.append(dur)

# Создание DataFrame с длительностями аудио
audio_df = pd.DataFrame({
    'file_id': range(1, 449),
    'duration': durations
})

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(audio_df['duration'], bins=30, kde=True)
plt.title("Распределение длительности аудиофайлов")
plt.xlabel("Длительность (сек)")
plt.ylabel("Количество файлов")
plt.show()

print(f"Средняя длительность аудиофайла: {audio_df['duration'].mean():.2f} сек")
print(f"Медианная длительность аудиофайла: {audio_df['duration'].median():.2f} сек")
print(f"Минимальная длительность: {audio_df['duration'].min():.2f} сек")
print(f"Максимальная длительность: {audio_df['duration'].max():.2f} сек")
print(f"Общая длительность всех аудиофайлов: {audio_df['duration'].sum()/3600:.2f} часов")

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(all_rttm['duration'], bins=50)
plt.title("Распределение длительности реплик")
plt.xlabel("Длительность (сек)")
plt.ylabel("Количество сегментов")
plt.show()

print(f"Средняя длительность реплики: {all_rttm['duration'].mean():.2f} сек")
print(f"Медианная длительность реплики: {all_rttm['duration'].median():.2f} сек")
print(f"Минимальная длительность реплики: {all_rttm['duration'].min():.2f} сек")
print(f"Максимальная длительность реплики: {all_rttm['duration'].max():.2f} сек")

In [ ]:
speaker_duration = all_rttm.groupby('name')['duration'].sum().sort_values(ascending=False)

plt.figure(figsize=(12, 6))
speaker_duration.plot(kind='bar')
plt.title("Общее время речи спикеров")
plt.xlabel("Спикер")
plt.ylabel("Суммарная длительность (сек)")
plt.xticks(rotation=45)
plt.show()

print(f"Общее количество уникальных спикеров: {len(speaker_duration)}")
print(f"Спикер с наибольшим временем речи: {speaker_duration.idxmax()} ({speaker_duration.max():.2f} сек)")
print(f"Спикер с наименьшим временем речи: {speaker_duration.idxmin()} ({speaker_duration.min():.2f} сек)")

In [ ]:
segments_per_file = all_rttm.groupby('file_id').size()

plt.figure(figsize=(12, 6))
sns.histplot(segments_per_file, bins=30)
plt.title("Количество реплик на файл")
plt.xlabel("Количество реплик")
plt.ylabel("Количество файлов")
plt.show()

print(f"Среднее количество реплик на файл: {segments_per_file.mean():.2f}")
print(f"Максимальное количество реплик в одном файле: {segments_per_file.max()}")
print(f"Минимальное количество реплик в одном файле: {segments_per_file.min()}")

In [ ]:
# Выберем несколько случайных файлов для визуализации
sample_files = all_rttm['file_id'].sample(5).unique()

plt.figure(figsize=(15, 10))
for i, file_id in enumerate(sample_files, 1):
    file_data = all_rttm[all_rttm['file_id'] == file_id]
    plt.subplot(5, 1, i)
    for _, row in file_data.iterrows():
        plt.plot([row['start'], row['end']], [row['name'], row['name']], marker='o')
    plt.title(f"Распределение реплик по времени (файл {file_id})")
    plt.xlabel("Время (сек)")
    plt.ylabel("Спикер")
plt.tight_layout()
plt.show()

In [ ]:
# Создаем сводную таблицу с характеристиками каждого файла
file_stats = pd.DataFrame({
    'duration': audio_df['duration'],
    'num_speakers': speakers_per_file.reset_index()['name'],
    'num_segments': segments_per_file.reset_index()[0]
}).reset_index()

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.scatterplot(data=file_stats, x='duration', y='num_speakers')
plt.title("Длительность аудио vs Количество спикеров")

plt.subplot(1, 2, 2)
sns.scatterplot(data=file_stats, x='duration', y='num_segments')
plt.title("Длительность аудио vs Количество реплик")
plt.tight_layout()
plt.show()

# Вычисление коэффициентов корреляции
corr_speakers = file_stats['duration'].corr(file_stats['num_speakers'])
corr_segments = file_stats['duration'].corr(file_stats['num_segments'])

print(f"Корреляция между длительностью и количеством спикеров: {corr_speakers:.2f}")
print(f"Корреляция между длительностью и количеством реплик: {corr_segments:.2f}")

In [ ]:
# Функция для вычисления пауз между репликами в файле
def calculate_pauses(file_data):
    file_data = file_data.sort_values('start')
    pauses = []
    for i in range(1, len(file_data)):
        prev_end = file_data.iloc[i-1]['end']
        curr_start = file_data.iloc[i]['start']
        pause = curr_start - prev_end
        if pause > 0:  # Исключаем перекрывающиеся реплики
            pauses.append(pause)
    return pauses

# Собираем паузы для всех файлов
all_pauses = []
for file_id in all_rttm['file_id'].unique():
    file_data = all_rttm[all_rttm['file_id'] == file_id]
    pauses = calculate_pauses(file_data)
    all_pauses.extend(pauses)

# Визуализация распределения пауз
plt.figure(figsize=(12, 6))
sns.histplot(all_pauses, bins=50)
plt.title("Распределение длительности пауз между репликами")
plt.xlabel("Длительность паузы (сек)")
plt.ylabel("Количество пауз")
plt.show()

print(f"Средняя длительность паузы: {pd.Series(all_pauses).mean():.2f} сек")
print(f"Медианная длительность паузы: {pd.Series(all_pauses).median():.2f} сек")
print(f"Минимальная пауза: {pd.Series(all_pauses).min():.2f} сек")
print(f"Максимальная пауза: {pd.Series(all_pauses).max():.2f} сек")

In [ ]:
print(f"Среднее количество спикеров на файл: {speakers_per_file.mean():.2f}")
print(f"Максимальное количество спикеров в одном файле: {speakers_per_file.max()}")
print(f"Минимальное количество спикеров в одном файле: {speakers_per_file.min()}")
print(f"Наиболее частое количество спикеров: {speakers_per_file.mode().values[0]}")

print(f"Средняя длительность аудиофайла: {audio_df['duration'].mean():.2f} сек")
print(f"Медианная длительность аудиофайла: {audio_df['duration'].median():.2f} сек")
print(f"Минимальная длительность: {audio_df['duration'].min():.2f} сек")
print(f"Максимальная длительность: {audio_df['duration'].max():.2f} сек")
print(f"Общая длительность всех аудиофайлов: {audio_df['duration'].sum()/3600:.2f} часов")

print(f"Средняя длительность реплики: {all_rttm['duration'].mean():.2f} сек")
print(f"Медианная длительность реплики: {all_rttm['duration'].median():.2f} сек")
print(f"Минимальная длительность реплики: {all_rttm['duration'].min():.2f} сек")
print(f"Максимальная длительность реплики: {all_rttm['duration'].max():.2f} сек")

print(f"Общее количество уникальных спикеров: {len(speaker_duration)}")
print(f"Спикер с наибольшим временем речи: {speaker_duration.idxmax()} ({speaker_duration.max():.2f} сек)")
print(f"Спикер с наименьшим временем речи: {speaker_duration.idxmin()} ({speaker_duration.min():.2f} сек)")

print(f"Среднее количество реплик на файл: {segments_per_file.mean():.2f}")
print(f"Максимальное количество реплик в одном файле: {segments_per_file.max()}")
print(f"Минимальное количество реплик в одном файле: {segments_per_file.min()}")

print(f"Корреляция между длительностью и количеством спикеров: {corr_speakers:.2f}")
print(f"Корреляция между длительностью и количеством реплик: {corr_segments:.2f}")


print(f"Средняя длительность паузы: {pd.Series(all_pauses).mean():.2f} сек")
print(f"Медианная длительность паузы: {pd.Series(all_pauses).median():.2f} сек")
print(f"Минимальная пауза: {pd.Series(all_pauses).min():.2f} сек")
print(f"Максимальная пауза: {pd.Series(all_pauses).max():.2f} сек")

# **Data Preprocessing** 

In [ ]:
import glob
from pathlib import Path
from tqdm import tqdm
import librosa
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
def parse_rttm(rttm_path):
    columns = ["type", "file_id", "channel", "start", "duration", 
               "ortho", "stype", "speaker_id", "conf", "slat"]
    data = []
    with open(rttm_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 8:
                continue
            data.append({
                "file_id": parts[1],
                "start": float(parts[3]),
                "duration": float(parts[4]),
                "speaker_id": parts[7]
            })
    return pd.DataFrame(data)

In [ ]:
def get_audio_duration(audio_path):
    duration = librosa.get_duration(filename=audio_path)
    return duration


In [ ]:
def build_dataset(audio_dir, rttm_dir):
    audio_files = list(Path(audio_dir).glob("*.wav"))  # или .mp3
    dataset = []
    
    for audio_path in tqdm(audio_files):
        file_id = audio_path.stem  # file1.wav -> file1
        duration = get_audio_duration(audio_path)
        
        # Парсим соответствующий .rttm
        rttm_path = Path(rttm_dir) / f"{file_id}.rttm"
        if not rttm_path.exists():
            continue
        
        df_rttm = parse_rttm(rttm_path)
        num_speakers = df_rttm["speaker_id"].nunique()
        num_segments = len(df_rttm)
        
        dataset.append({
            "file_id": file_id,
            "duration": duration,
            "num_speakers": num_speakers,
            "num_segments": num_segments,
            "speakers": list(df_rttm["speaker_id"].unique()),
            "audio_path": str(audio_path),
            "rttm_path": str(rttm_path)
        })
    
    return pd.DataFrame(dataset)


audio_dir = "/kaggle/input/diarization/Dataset/data/audio"
rttm_dir = "/kaggle/input/diarization/Dataset/data/markups"

df = build_dataset(audio_dir, rttm_dir)
df.to_csv("audio_dataset.csv", sep =';', index=False)  

In [ ]:

df = pd.read_csv("/kaggle/working/audio_dataset.csv", sep=';')



In [ ]:
df.head(3)

In [ ]:
df_balanced = df[(df['num_speakers'] >= 2) & (df['num_speakers'] <= 7)].copy()

# Разделение длинных файлов (пример для файлов >600 сек)
max_duration = 600
df_long = df_balanced[df_balanced['duration'] > max_duration]
df_balanced = df_balanced[df_balanced['duration'] <= max_duration]

# Для каждого длинного файла создаем сегменты (примерно по 300 сек)
segments = []
for _, row in df_long.iterrows():
    num_segments = int(row['duration'] // max_duration) + 1
    for i in range(num_segments):
        segment = row.copy()
        segment['duration'] = max_duration if i < num_segments - 1 else row['duration'] % max_duration
        segments.append(segment)

# Добавляем сегменты обратно в датасет
df_balanced = pd.concat([df_balanced, pd.DataFrame(segments)], ignore_index=True)

# Балансировка по количеству спикеров (например, оставляем 100 файлов для каждой группы)
min_samples = 100
df_balanced = df_balanced.groupby('num_speakers').apply(lambda x: x.sample(min(min_samples, len(x))).reset_index(drop=True))

# Сохранение сбалансированного датасета
df_balanced.to_csv('balanced_audio_dataset.csv', sep=';', index=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Распределение количества спикеров
plt.figure(figsize=(10, 6))
sns.countplot(data=df_balanced, x='num_speakers')
plt.title('Распределение количества спикеров после балансировки')
plt.xlabel('Количество спикеров')
plt.ylabel('Количество файлов')
plt.show()

# 2. Распределение длительности файлов
plt.figure(figsize=(10, 6))
sns.histplot(df_balanced['duration'], bins=30)
plt.title('Распределение длительности аудиофайлов')
plt.xlabel('Длительность (сек)')
plt.ylabel('Количество файлов')
plt.axvline(df_balanced['duration'].mean(), color='r', linestyle='--', label=f'Среднее: {df_balanced["duration"].mean():.1f} сек')
plt.legend()
plt.show()

# 3. Соотношение количества спикеров и длительности
plt.figure(figsize=(10, 6))
sns.boxplot(data=df_balanced, x='num_speakers', y='duration')
plt.title('Распределение длительности по количеству спикеров')
plt.xlabel('Количество спикеров')
plt.ylabel('Длительность (сек)')
plt.show()

# 4. Распределение количества реплик
plt.figure(figsize=(10, 6))
sns.histplot(df_balanced['num_segments'], bins=30)
plt.title('Распределение количества реплик в файлах')
plt.xlabel('Количество реплик')
plt.ylabel('Количество файлов')
plt.axvline(df_balanced['num_segments'].mean(), color='r', linestyle='--', label=f'Среднее: {df_balanced["num_segments"].mean():.1f}')
plt.legend()
plt.show()

# 5. Соотношение количества спикеров и реплик
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_balanced, x='num_speakers', y='num_segments', alpha=0.6)
plt.title('Соотношение количества спикеров и реплик')
plt.xlabel('Количество спикеров')
plt.ylabel('Количество реплик')
plt.show()

In [ ]:
print(f"Общая длительность всех аудиофайлов: {df_balanced['duration'].sum()/3600:.2f} часов")

# **Dataset**

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import soundfile as sf
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [ ]:
class EENDDataset(Dataset):
    def __init__(self, csv_file, frame_size=512, frame_shift=256, chunk_size=2000, hop_size=2000, max_speakers=7, transform=None):
        self.data = pd.read_csv(csv_file, sep=';')
        self.frame_size = frame_size
        self.frame_shift = frame_shift
        self.chunk_size = chunk_size
        self.hop_size = hop_size if hop_size is not None else chunk_size // 2
        self.max_speakers = max_speakers
        self.transform = transform
        self.label_encoder = LabelEncoder()
        
        all_speakers = []
        for speakers in self.data['speakers']:
            all_speakers.extend(eval(speakers))
        self.label_encoder.fit(list(set(all_speakers)) + ['overlap', 'silence'])
        
        self.chunk_indices = self._generate_chunk_indices()
    
    def _generate_chunk_indices(self):
        chunk_indices = []
        for idx in range(len(self.data)):
            row = self.data.iloc[idx]
            audio_path = row['audio_path']
            
            audio, sr = sf.read(audio_path)
            if len(audio.shape) > 1:
                audio = audio.mean(axis=1)
                
            num_frames = (len(audio) - self.frame_size) // self.frame_shift + 1
            num_chunks = (num_frames - self.chunk_size) // self.hop_size + 1
            
            for i in range(num_chunks):
                start_frame = i * self.hop_size
                end_frame = start_frame + self.chunk_size
                chunk_indices.append((idx, start_frame, end_frame))
        print(f"Создано {len(chunk_indices)} чанков ")        
        return chunk_indices
    
    def __len__(self):
        return len(self.chunk_indices)
    
    def __getitem__(self, chunk_idx):
        file_idx, start_frame, end_frame = self.chunk_indices[chunk_idx]
        row = self.data.iloc[file_idx]
        audio_path = row['audio_path']
        rttm_path = row['rttm_path']
        num_speakers = row['num_speakers']

        audio, sr = sf.read(audio_path)
        if len(audio.shape) > 1:
            audio = audio.mean(axis=1)

     
        mel_spec = librosa.feature.melspectrogram(
            y=audio,
            sr=sr,
            n_fft=self.frame_size,
            hop_length=self.frame_shift,
            n_mels=80   
        )
        mel_spec = librosa.power_to_db(mel_spec, ref=np.max)   
        mel_spec = torch.FloatTensor(mel_spec.T)  

     
        start_frame = max(0, min(start_frame, mel_spec.shape[0] - self.chunk_size))
        end_frame = start_frame + self.chunk_size
        mel_spec = mel_spec[start_frame:end_frame]

         
        if mel_spec.shape[0] < self.chunk_size:
            pad_len = self.chunk_size - mel_spec.shape[0]
            mel_spec = torch.cat([mel_spec, torch.zeros(pad_len, 80)], dim=0)

        labels = self.parse_rttm(rttm_path, len(audio), sr)
        labels = labels[start_frame:end_frame]

        sample = {
            'features': mel_spec,   
            'labels': torch.FloatTensor(labels),
            'num_speakers': num_speakers
        }
        return sample
    
    def parse_rttm(self, rttm_path, audio_length, sr):
        num_frames = (audio_length - self.frame_size) // self.frame_shift + 1
        frame_duration = self.frame_size / sr
        
        labels = np.zeros((num_frames, self.max_speakers), dtype=np.float32)
        
        try:
            with open(rttm_path, 'r') as f:
                lines = f.readlines()
        except FileNotFoundError:
            return labels
            
        speaker_mapping = {}
        
        for line in lines:
            parts = line.strip().split()
            if len(parts) < 9:
                continue
                
            speaker_id = parts[7]
            start_time = float(parts[3])
            duration = float(parts[4])
            end_time = start_time + duration
            
            start_frame = int(start_time / frame_duration)
            end_frame = int(end_time / frame_duration)
            
            start_frame = max(0, min(start_frame, num_frames - 1))
            end_frame = max(0, min(end_frame, num_frames - 1))
            
            if speaker_id not in speaker_mapping:
                if len(speaker_mapping) >= self.max_speakers:
                    continue
                speaker_mapping[speaker_id] = len(speaker_mapping)
            
            spk_idx = speaker_mapping[speaker_id]
            labels[start_frame:end_frame+1, spk_idx] = 1
            
        overlap = np.sum(labels, axis=1) > 1
        if np.any(overlap) and len(speaker_mapping) < self.max_speakers:
            overlap_channel = len(speaker_mapping)
            if overlap_channel < self.max_speakers:
                labels[overlap, overlap_channel] = 1
                for spk_idx in range(len(speaker_mapping)):
                    labels[overlap, spk_idx] = 0
                    
        return labels

# **Model**

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer


In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model=512):
        super().__init__()
        self.d_model = d_model

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        batch_size, seq_len, _ = x.shape
        pe = torch.zeros(seq_len, self.d_model, device=x.device)

        position = torch.arange(seq_len, device=x.device).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, self.d_model, 2, device=x.device) * (-math.log(10000.0) / self.d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        return x + pe.unsqueeze(0)

In [ ]:
class Config:
    sample_rate = 16000
    feature_dim = 80  # MFCC features
    hidden_size = 256
    num_layers = 4
    max_speakers = 7  # Максимальное количество спикеров в датасете
    dropout = 0.1
    learning_rate = 0.0001
    batch_size = 8
    epochs = 50
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_save_path = "best_eend_model.pth"
    metrics_save_path = "training_metrics.json"
    max_audio_length = 600  # Максимальная длина аудио в секундах (10 минут)
    segment_length = 30  # Длина сегмента в секундах для батчинга
    overlap = 5  # Перекрытие сегментов в секундах

# Модель EEND
class EENDModel(nn.Module):
    def __init__(self ):
        super(EENDModel, self).__init__()
        config = Config()
        
        # Encoder для обработки аудио признаков
        self.encoder = nn.LSTM(
            input_size=config.feature_dim,
            hidden_size=config.hidden_size,
            num_layers=config.num_layers,
            dropout=config.dropout if config.num_layers > 1 else 0,
            bidirectional=True,
            batch_first=True
        )
        
        # Speaker prediction layers
        self.speaker_layers = nn.ModuleList([
            nn.Sequential(
                nn.Linear(2 * config.hidden_size, config.hidden_size),
                nn.ReLU(),
                nn.Linear(config.hidden_size, 1),
                nn.Sigmoid()
            ) for _ in range(config.max_speakers)
        ])
        
    def forward(self, x):
        # x: (batch_size, seq_len, feature_dim)
        outputs, _ = self.encoder(x)  # (batch_size, seq_len, 2*hidden_size)
        
        # Для каждого потенциального спикера предсказываем вероятность его присутствия
        speaker_probs = [layer(outputs) for layer in self.speaker_layers]
        speaker_probs = torch.stack(speaker_probs, dim=-1)  # (batch_size, seq_len, max_speakers)
        
        return speaker_probs

# **Loss**

In [ ]:
def permutation_invariant_loss(outputs, targets, num_speakers):
    """Улучшенная версия PIT loss с автоматической корректировкой размерностей"""
    # Проверка размерностей
    if outputs.dim() != targets.dim():
        if outputs.dim() == 4 and targets.dim() == 3:
            outputs = outputs.squeeze(1)  # Удаляем dimension каналов [B,C,T,S] -> [B,T,S]
        else:
            raise ValueError(f"Dimension mismatch: outputs {outputs.shape}, targets {targets.shape}")
    
    batch_size, seq_len, max_speakers = outputs.shape
    total_loss = 0.0
    
    for i in range(batch_size):
        n_spk = num_speakers[i].item() if torch.is_tensor(num_speakers) else num_speakers
        output = outputs[i, :, :n_spk]
        target = targets[i, :, :n_spk]
        
        # Быстрая проверка перестановок (2 варианта)
        if n_spk == 1:
            loss = F.binary_cross_entropy(output, target, reduction='sum')
        else:
            loss1 = F.binary_cross_entropy(output, target, reduction='sum')
            loss2 = F.binary_cross_entropy(output.flip(-1), target, reduction='sum')
            loss = min(loss1, loss2)
        
        total_loss += loss / seq_len
    
    return total_loss / batch_size

# **Training**

In [ ]:
from tqdm import tqdm
import json
import os
import numpy as np
from itertools import permutations
from sklearn.metrics import confusion_matrix
import math
import itertools

In [52]:
def calculate_der(preds, labels, num_speakers):
     
    preds = preds > 0.5   
    labels = labels.bool()
    
    total_errors = 0
    total_frames = 0
    
    for i in range(len(preds)):
        n = int(num_speakers[i].item())
        pred = preds[i, :, :n]
        target = labels[i, :, :n]
        
         
        best_error = float('inf')
        for perm in permutations(range(n)):
            permuted_pred = pred[:, perm]
            
            tn, fp, fn, tp = confusion_matrix(
                target.cpu().numpy().flatten(),
                permuted_pred.cpu().numpy().flatten(),
                labels=[0, 1]
            ).ravel()
            
            current_error = fp + fn   
            if current_error < best_error:
                best_error = current_error
        
        total_errors += best_error
        total_frames += target.numel()
    
    return total_errors / total_frames if total_frames > 0 else 0.0

In [ ]:
def train():   
    
    model = EENDEDA(
        input_dim=80,
        hidden_dim=384,
        num_layers=4,
        n_speakers=21,
        dropout=0.3
    ).to(device)

    
    optimizer = optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-5)
    criterion = AdaptiveFocalLoss(n_speakers=21).to(device)
    metrics = DiarizationMetrics(max_speakers=21)
    
    history = {
        'train_loss': [],
        'val_loss': [],
        'DER': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'lr': []
    }
    
    best_der = float('inf')
    early_stop_patience = 5
    early_stop_counter = 0

    
    for epoch in range(100):
        model.train()
        metrics.reset()
        total_loss = 0
        
         
        curr_length = min(15 + epoch//5, 30)
        dataset.chunk_size = curr_length
        
        for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}", total=len(dataloader)):
            features, labels = batch
            features = features.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            
            optimizer.zero_grad()
            outputs = model(features)          
            
            spk_presence = labels.sum((0,1)) > 0
            active_outputs = outputs[..., spk_presence]
            active_targets = labels[..., spk_presence]
            
            loss = criterion(active_outputs, active_targets)
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            total_loss += loss.item()
            metrics.update(outputs.sigmoid().detach().cpu().numpy(), 
                         labels.cpu().numpy())

         
        epoch_metrics = metrics.get_metrics()
        epoch_loss = total_loss / len(dataloader)
        
        
        history['train_loss'].append(epoch_loss)
        history['DER'].append(epoch_metrics['DER'])
        history['precision'].append(epoch_metrics['Precision'])
        history['recall'].append(epoch_metrics['Recall'])
        history['f1'].append(epoch_metrics['F1'])
        history['lr'].append(optimizer.param_groups[0]['lr'])
        
        
        print(f"\nEpoch {epoch+1}:")
        print(f"Loss: {epoch_loss:.4f} | DER: {epoch_metrics['DER']:.4f}")
        print(f"Precision: {epoch_metrics['Precision']:.4f} | Recall: {epoch_metrics['Recall']:.4f}")
        print(f"F1: {epoch_metrics['F1']:.4f} | LR: {optimizer.param_groups[0]['lr']:.2e}")

         
        if epoch_metrics['DER'] < best_der:
            best_der = epoch_metrics['DER']
            early_stop_counter = 0
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'metrics': epoch_metrics,
                'history': history
            }, "best_model.pth")
        else:
            early_stop_counter += 1
            if early_stop_counter >= early_stop_patience:
                print(f"\nEarly stopping at epoch {epoch+1}")
                break

     
    plot_training_history(history)
    
    return history, model

In [ ]:
 def plot_training_history(history):
    """Визуализация метрик обучения"""
    plt.figure(figsize=(15, 5))
    
    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.title('Training Loss')
    plt.xlabel('Epoch')
    plt.grid(True)
    
    # Метрики
    plt.subplot(1, 2, 2)
    plt.plot(history['DER'], label='DER')
    plt.plot(history['f1'], label='F1')
    plt.title('Performance Metrics')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()

In [ ]:
 if __name__ == "__main__":
    res = train()